In [ ]:
# 명함인식 구현하기

import numpy as np
import cv2

def order_points(pts):
    rect = np.zeros((4,2), dtype = 'float32')
    
    S = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(S)]
    rect[2] = pts[np.argmax(S)]
    
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    
    return rect

def auto_scan_image_via_webcam():
    try:
        cap = cv2.VideoCapture(0)
    except:
        print('cannot load camera!')
        return
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print('cannot load camera!')
            break
        k = cv2.waitKey(10)
        if k == 27:
            break
            
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (3,3), 0)
        edged = cv2.Canny(gray, 75, 200)
        
        print("STEP 1: Edge Detection")
    
        (cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
    
        for c in cnts:
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02 * peri, True)
            screenCnt = []
            
            if len(approx) == 4:
                contourSize = cv2.contourArea(approx)
                camSize = frame.shape[0] * frame.shape[1]
                ratio = contourSize / camSize
                print(contourSize)
                print(camSize)
                print(ratio)
            
                if ratio > 0.1:
                    screenCnt = approx
            
                break
                
        if len(screenCnt) == 0:
            cv2.imshow("WebCam", frame)
            continue
        
        else:
            print("STEP 2: Find contours of paper")
        
            cv2.drawContours(frame, [screenCnt], -1, (0,255,0), 2)
            cv2.imshow("WebCam", frame)
        
            rect = order_points(screenCnt.reshape(4,2))
            (topLeft, topRight, bottomRight, bottomLeft) = rect
        
            w1 = abs(bottomRight[0] - bottomLeft[0])
            w2 = abs(topRight[0] - topLeft[0])
            h1 = abs(topRight[1] - bottomRight[1])
            h2 = abs(topLeft[1] - bottomLeft[1])
            maxWidth = max([w1, w2])
            maxHeight = max([h1, h2])
        
            dst = np.float32([[0,0], [maxWidth-1,0], [maxWidth-1, maxHeight-1], [0,maxHeight-1]])
        
            M = cv2.getPerspectiveTransform(rect, dst)
            warped = cv2.warpPerspective(frame, M, (maxWidth, maxHeight))
        
            print("STEP 3: Apply perspective transform")
        
            warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
            warped = cv2.adaptiveThreshold(warped, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 10)
        
            print("STEP 4: Apply Adaptive Threshold")
        
            break
        
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    
    cv2.imshow("Scanned", warped)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    
if __name__ == '__main__':
    auto_scan_image_via_webcam()

STEP 1: Edge Detection
792.0
307200
0.002578125
STEP 1: Edge Detection
STEP 1: Edge Detection
STEP 1: Edge Detection
629.0
307200
0.0020475260416666667
STEP 1: Edge Detection
629.0
307200
0.0020475260416666667
STEP 1: Edge Detection
STEP 1: Edge Detection
STEP 1: Edge Detection
521.0
307200
0.0016959635416666666
STEP 1: Edge Detection
527.5
307200
0.0017171223958333333
STEP 1: Edge Detection
560.0
307200
0.0018229166666666667
STEP 1: Edge Detection
STEP 1: Edge Detection
STEP 1: Edge Detection
STEP 1: Edge Detection
567.0
307200
0.001845703125
STEP 1: Edge Detection
513.0
307200
0.001669921875
STEP 1: Edge Detection
528.0
307200
0.00171875
STEP 1: Edge Detection
STEP 1: Edge Detection
486.0
307200
0.00158203125
STEP 1: Edge Detection
474.0
307200
0.00154296875
STEP 1: Edge Detection
503.0
307200
0.0016373697916666667
STEP 1: Edge Detection
STEP 1: Edge Detection
STEP 1: Edge Detection
522.5
307200
0.0017008463541666667
STEP 1: Edge Detection
495.0
307200
0.001611328125
STEP 1: Edge Det